In [16]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

# Methods - Text Feature Extraction with Bag-of-Words

In many tasks, like in the classical spam detection, your input data is text.
Free text with variables length is very far from the fixed length numeric representation that we need to do machine learning with scikit-learn.
However, there is an easy and effective way to go from text data to a numeric representation using the so-called bag-of-words model, which provides a data structure that is compatible with the machine learning aglorithms in scikit-learn.

<img src="figures/bag_of_words.svg" width="100%">


Let's assume that each sample in your dataset is represented as one string, which could be just a sentence, an email, or a whole news article or book. To represent the sample, we first split the string into a list of tokens, which correspond to (somewhat normalized) words. A simple way to do this to just split by whitespace, and then lowercase the word. 

Then, we build a vocabulary of all tokens (lowercased words) that appear in our whole dataset. This is usually a very large vocabulary.
Finally, looking at our single sample, we could show how often each word in the vocabulary appears.
We represent our string by a vector, where each entry is how often a given word in the vocabulary appears in the string.

As each sample will only contain very few words, most entries will be zero, leading to a very high-dimensional but sparse representation.

The method is called "bag-of-words," as the order of the words is lost entirely.

In [17]:
X = ["Some say the world will end in fire,",
     "Some say in ice."]

In [18]:
len(X)

2

In [19]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(X)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [20]:
vectorizer.vocabulary_

{'some': 5,
 'say': 4,
 'the': 6,
 'world': 8,
 'will': 7,
 'end': 0,
 'in': 3,
 'fire': 1,
 'ice': 2}

In [21]:
X_bag_of_words = vectorizer.transform(X)

In [22]:
X_bag_of_words.shape

(2, 9)

In [23]:
X_bag_of_words

<2x9 sparse matrix of type '<class 'numpy.int64'>'
	with 12 stored elements in Compressed Sparse Row format>

In [24]:
X_bag_of_words.toarray()

array([[1, 1, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 0, 0, 0]], dtype=int64)

In [25]:
vectorizer.get_feature_names()

['end', 'fire', 'ice', 'in', 'say', 'some', 'the', 'will', 'world']

In [26]:
vectorizer.inverse_transform(X_bag_of_words)

[array(['end', 'fire', 'in', 'say', 'some', 'the', 'will', 'world'],
       dtype='<U5'), array(['ice', 'in', 'say', 'some'], dtype='<U5')]

# tf-idf Encoding
A useful transformation that is often applied to the bag-of-word encoding is the so-called term-frequency inverse-document-frequency (tf-idf) scaling, which is a non-linear transformation of the word counts.

The tf-idf encoding rescales words that are common to have less weight:

In [27]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(X)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [28]:
import numpy as np
np.set_printoptions(precision=2)

print(tfidf_vectorizer.transform(X).toarray())

[[0.39 0.39 0.   0.28 0.28 0.28 0.39 0.39 0.39]
 [0.   0.   0.63 0.45 0.45 0.45 0.   0.   0.  ]]


tf-idfs are a way to represent documents as feature vectors. tf-idfs can be understood as a modification of the raw term frequencies (`tf`); the `tf` is the count of how often a particular word occurs in a given document. The concept behind the tf-idf is to downweight terms proportionally to the number of documents in which they occur. Here, the idea is that terms that occur in many different documents are likely unimportant or don't contain any useful information for Natural Language Processing tasks such as document classification. If you are interested in the mathematical details and equations, see this [external IPython Notebook](http://nbviewer.jupyter.org/github/rasbt/pattern_classification/blob/master/machine_learning/scikit-learn/tfidf_scikit-learn.ipynb) that walks you through the computation.

# Bigrams and N-Grams

In the example illustrated in the figure at the beginning of this notebook, we used the so-called 1-gram (unigram) tokenization: Each token represents a single element with regard to the splittling criterion. 

Entirely discarding word order is not always a good idea, as composite phrases often have specific meaning, and modifiers like "not" can invert the meaning of words.

A simple way to include some word order are n-grams, which don't only look at a single token, but at all pairs of neighborhing tokens. For example, in 2-gram (bigram) tokenization, we would group words together with an overlap of one word; in 3-gram (trigram) splits we would create an overlap two words, and so forth:

- original text: "this is how you get ants"
- 1-gram: "this", "is", "how", "you", "get", "ants"
- 2-gram: "this is", "is how", "how you", "you get", "get ants"
- 3-gram: "this is how", "is how you", "how you get", "you get ants"

Which "n" we choose for "n-gram" tokenization to obtain the optimal performance in our predictive model depends on the learning algorithm, dataset, and task. Or in other words, we have consider "n" in "n-grams" as a tuning parameters, and in later notebooks, we will see how we deal with these.

Now, let's create a bag of words model of bigrams using scikit-learn's `CountVectorizer`:

In [29]:
# look at sequences of tokens of minimum length 2 and maximum length 2
bigram_vectorizer = CountVectorizer(ngram_range=(2, 2))
bigram_vectorizer.fit(X)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [30]:
bigram_vectorizer.get_feature_names()

['end in',
 'in fire',
 'in ice',
 'say in',
 'say the',
 'some say',
 'the world',
 'will end',
 'world will']

In [31]:
bigram_vectorizer.transform(X).toarray()

array([[1, 1, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 0, 1, 0, 0, 0]], dtype=int64)

Often we want to include unigrams (single tokens) AND bigrams, wich we can do by passing the following tuple as an argument to the `ngram_range` parameter of the `CountVectorizer` function:

In [32]:
gram_vectorizer = CountVectorizer(ngram_range=(1, 2))
gram_vectorizer.fit(X)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [33]:
gram_vectorizer.get_feature_names()

['end',
 'end in',
 'fire',
 'ice',
 'in',
 'in fire',
 'in ice',
 'say',
 'say in',
 'say the',
 'some',
 'some say',
 'the',
 'the world',
 'will',
 'will end',
 'world',
 'world will']

In [34]:
gram_vectorizer.transform(X).toarray()

array([[1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Character n-grams
=================

Sometimes it is also helpful not only to look at words, but to consider single characters instead.   
That is particularly useful if we have very noisy data and want to identify the language, or if we want to predict something about a single word.
We can simply look at characters instead of words by setting ``analyzer="char"``.
Looking at single characters is usually not very informative, but looking at longer n-grams of characters could be:

In [35]:
X

['Some say the world will end in fire,', 'Some say in ice.']

In [36]:
char_vectorizer = CountVectorizer(ngram_range=(2, 2), analyzer="char")
char_vectorizer.fit(X)

CountVectorizer(analyzer='char', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [37]:
print(char_vectorizer.get_feature_names())

[' e', ' f', ' i', ' s', ' t', ' w', 'ay', 'ce', 'd ', 'e ', 'e,', 'e.', 'en', 'fi', 'he', 'ic', 'il', 'in', 'ir', 'l ', 'ld', 'll', 'me', 'n ', 'nd', 'om', 'or', 're', 'rl', 'sa', 'so', 'th', 'wi', 'wo', 'y ']


<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>
      Compute the bigrams from "zen of python" as given below (or by ``import this``), and find the most common trigram.
We want to treat each line as a separate document. You can achieve this by splitting the string by newlines (``\n``).
Compute the Tf-idf encoding of the data. Which words have the highest tf-idf score? Why?
What changes if you use ``TfidfVectorizer(norm="none")``?
      </li>
    </ul>
</div>

In [38]:
zen = """Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!"""

## BMC solution

In [39]:
text = zen.split("\n")
print(len(text))
print(text[5])

19
Sparse is better than dense.


In [40]:
from sklearn.feature_extraction.text import CountVectorizer #import estimator


# look at sequences of tokens of minimum length 2 and maximum length 2
bigram_vectorizer2 = CountVectorizer(ngram_range=(2, 2)) #instantiate
bigram_vectorizer2.fit(text) #fit estimator

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(2, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [41]:
featureNames = bigram_vectorizer2.get_feature_names()
bigram_vectorizer2.get_feature_names()

['although never',
 'although practicality',
 'although that',
 'ambiguity refuse',
 'and preferably',
 'are one',
 'aren special',
 'at first',
 'bad idea',
 'be good',
 'be obvious',
 'be one',
 'beats purity',
 'beautiful is',
 'better than',
 'break the',
 'cases aren',
 'complex is',
 'do it',
 'do more',
 'easy to',
 'enough to',
 'errors should',
 'explain it',
 'explicit is',
 'explicitly silenced',
 'face of',
 'first unless',
 'flat is',
 'good idea',
 'great idea',
 'hard to',
 'honking great',
 'idea let',
 'if the',
 'implementation is',
 'in the',
 'is better',
 'is easy',
 'is hard',
 'is often',
 'it bad',
 'it may',
 'let do',
 'may be',
 'may not',
 'more of',
 'namespaces are',
 'never is',
 'never pass',
 'not be',
 'now is',
 'obvious at',
 'obvious way',
 'of ambiguity',
 'of those',
 'often better',
 'one and',
 'one honking',
 'one obvious',
 'only one',
 'pass silently',
 'practicality beats',
 'preferably only',
 're dutch',
 'readability counts',
 'refuse the

In [45]:
print(bigram_vectorizer2.transform(text).toarray().shape)
print(bigram_vectorizer2.transform(text).toarray().sum(axis=0)) #find maximum word counts

maxes = bigram_vectorizer2.transform(text).toarray().sum(axis=0)

max_pos = np.where(maxes == np.amax(maxes))

featureNames = np.array(featureNames)

print(featureNames[max_pos])

print("")
print("Solution 1: The most common bi-gram is {}".format(featureNames[max_pos]))


(19, 98)
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 8 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 2 2 1
 7 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 2 1 1 1 1 1 1]
['better than']

Solution 1: The most common bi-gram is ['better than']


In [77]:
## Compute the tf-idf encoding

from sklearn.feature_extraction.text import TfidfVectorizer

#tfidf_vectorizer2 = TfidfVectorizer(norm="l2")
tfidf_vectorizer2.fit(text)

import numpy as np
np.set_printoptions(precision=2)

print(tfidf_vectorizer2.transform(text).toarray().shape)
print(tfidf_vectorizer2.transform(text).toarray().max(axis=0)) #find maximum word counts

maxes2 = tfidf_vectorizer2.transform(text).toarray().max(axis=0)
maxes2 = tfidf_vectorizer2.transform(text).toarray().max(axis=0)

max_pos2 = np.where(maxes2 == np.amax(maxes2))
max_list = np.where(maxes2 > 0.6)

print("")
print(max_pos2)

featureNames2 = tfidf_vectorizer2.get_feature_names()

featureNames2 = np.array(featureNames2)

print(featureNames2[max_pos2])

print("")
print("Solution 2a: The highest tf-idf score is for {}".format(featureNames2[max_pos2]))
print("")
print("Solution 2b: The highest tf-idf scores are for {}".format(featureNames2[max_list]))




(19, 79)
[0.42 0.33 0.29 0.32 0.32 0.3  0.38 0.28 0.53 0.59 0.37 0.32 0.32 0.54
 0.62 0.71 0.59 0.28 0.3  0.35 0.32 0.48 0.34 0.59 0.6  0.33 0.3  0.59
 0.35 0.32 0.33 0.38 0.32 0.3  0.34 0.34 0.59 0.33 0.32 0.3  0.32 0.31
 0.32 0.32 0.59 0.53 0.3  0.59 0.26 0.29 0.45 0.51 0.29 0.48 0.53 0.29
 0.53 0.3  0.71 0.33 0.45 0.32 0.42 0.6  0.48 0.62 0.59 0.63 0.33 0.37
 0.3  0.47 0.29 0.32 0.26 0.59 0.53 0.26 0.3 ]

(array([15, 58], dtype=int64),)
['counts' 'readability']

Solution 2a: The highest tf-idf score is for ['counts' 'readability']

Solution 2b: The highest tf-idf scores are for ['complicated' 'counts' 'explicitly' 'readability' 'silenced' 'simple'
 'special']


In [65]:
tfidf_vectorizer2.get_feature_names()


['although',
 'ambiguity',
 'and',
 'are',
 'aren',
 'at',
 'bad',
 'be',
 'beats',
 'beautiful',
 'better',
 'break',
 'cases',
 'complex',
 'complicated',
 'counts',
 'dense',
 'do',
 'dutch',
 'easy',
 'enough',
 'errors',
 'explain',
 'explicit',
 'explicitly',
 'face',
 'first',
 'flat',
 'good',
 'great',
 'guess',
 'hard',
 'honking',
 'idea',
 'if',
 'implementation',
 'implicit',
 'in',
 'is',
 'it',
 'let',
 'may',
 'more',
 'namespaces',
 'nested',
 'never',
 'not',
 'now',
 'obvious',
 'of',
 'often',
 'one',
 'only',
 'pass',
 'practicality',
 'preferably',
 'purity',
 're',
 'readability',
 'refuse',
 'right',
 'rules',
 'should',
 'silenced',
 'silently',
 'simple',
 'sparse',
 'special',
 'temptation',
 'than',
 'that',
 'the',
 'there',
 'those',
 'to',
 'ugly',
 'unless',
 'way',
 'you']

## Solutions

In [67]:
# %load solutions/11_ngrams.py
text = zen.split("\n")
for n in [2, 3, 4]:
    cv = CountVectorizer(ngram_range=(n, n)).fit(text)
    counts = cv.transform(text)
    most_common = np.argmax(counts.sum(axis=0))
    print("most common %d-gram: %s" % (n, cv.get_feature_names()[most_common]))


for norm in ["l2", None]:
    tfidf_vect = TfidfVectorizer(norm=norm).fit(text)
    data_tfidf = tfidf_vect.transform(text) 
    most_common = tfidf_vect.get_feature_names()[np.argmax(data_tfidf.max(axis=0).toarray())]
    print("highest tf-idf with norm=%s: %s" % (norm, most_common))


most common 2-gram: better than
most common 3-gram: is better than
most common 4-gram: if the implementation is
highest tf-idf with norm=l2: counts
highest tf-idf with norm=None: special
